# MIS-CPIMS Data Import Scripts

Import pandas library and read exported CPARA data from MIS (XLSX). Display all columns for exploration.

In [3]:
import pandas as pd
# Reads USAID 4The Child cpara data
cpara=pd.read_excel("Data/CPARA.xlsx", sheet_name="Sheet1")
# cpara.columns

Index(['UnIque ID', 'record_id', 'assessment_date', 'cpims_id', 'case_manager',
       'child_headed', 'has_hei', 'has_pbf', 'has_svac', 'q1_1', 'q1_2',
       'q1_3', 'q1_4', 'benchmark1', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5',
       'q2_6', 'q2_8', 'q2_7', 'q2_9', 'benchmark2', 'q3_1', 'q3_2', 'q3_3',
       'benchmark3', 'q4_1', 'q4_2', 'q4_3', 'q4_4', 'benchmark4', 'q5_1',
       'q5_2', 'q5_3', 'benchmark5', 'q6_1', 'q6_2', 'q6_3', 'q6_4', 'q6_5',
       'benchmark6', 'q7_1', 'q7_2', 'benchmark7', 'q8_1', 'benchmark8',
       'q9_1', 'q9_2', 'q9_3', 'q9_4', 'benchmark9', 'healthy', 'stable',
       'safe', 'schooled', 'entry_time', 'action_by', 'designation', 'lip',
       'entry_by_name', 'client_time', 'deleted', 'modified', 'gps', 'device',
       'cpimsId', 'names', 'caregiverhivstatus', 'cbo_id', 'cbo', 'county',
       'subcounty', 'ward', 'chv_id', 'chv'],
      dtype='object')

In [ ]:
# reads the mapping for U4TC mapping for HH_id, cpims_ovc_id, caregiver id
cpims_hhs=pd.read_excel("Data/Household ID OVC Mapping.xlsx", sheet_name="Sheet1")

In [ ]:
# Reads individual cpara data for U4TC
cpara_ovc_questions = pd.read_excel("Data/OVC_Questions.xlsx", sheet_name="Sheet1")

In [6]:
# Generate UID whose length is equivalent to the U4TC cpara data
import uuid

record_counts = cpara['record_id'].count()

def generate_uid(count):
    uid_list = []
    for i in range (0, count):
        uuid_gen = uuid.uuid4()
        # print(f'generated uid: {uuid_gen} inte: {i}')
        uid_list.append(uuid_gen)
    return uid_list    

df_uid = pd.DataFrame(generate_uid(record_counts))

In [7]:
# Reads the cpims questions mapping
cpims_questions=pd.read_excel("Data/ovc_care_questions.xlsx", sheet_name="Sheet1")

In [27]:
# Mergers cpara data with household id, cpims id on caregiver_id
cpara_data1=pd.merge(cpara, cpims_hhs, how='left', left_on='cpims_id', right_on='caregiver_id')
# Merges cpara data with the uids
cpara_data = pd.merge(cpara_data1, df_uid, left_index=True, right_index=True)
cpara_data.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,cbo,county,subcounty,ward,chv_id,chv,cpims_ovc_id,caregiver_id,household,0
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282788.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,16142843-84ab-4cbe-9edd-de02ac81eef9
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282807.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,785a1f6b-6301-40dd-ac67-72b5f059d121


In [15]:
# cpara individual table merges with with household id, cpims id on caregiver_id
cpara_individual=pd.merge(cpara_ovc_questions, cpims_hhs, how='left', left_on='caregiver_id', right_on='caregiver_id')
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [5]:
cpara_ovc_questions.dtypes #cpims_hhs.dtypes

record_id           int64
cpara_record_id     int64
cpara_date         object
question           object
response           object
cpims_id            int64
caregiver_id        int64
chv_id              int64
dtype: object

Align the MIS data from rows to columns => vertical and only display the first two records. Move the columns that should not be considered as var.

In [19]:
# converts the horizontal table of cpara data to vertical table for manipulation
cpara_unpivot=pd.melt(cpara_data, id_vars=['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name','client_time', 'modified', 'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household', 0])

cpara_unpivot.head(2)

,record_id,assessment_date,cpims_id,entry_time,action_by,designation,lip,entry_by_name,client_time,modified,deleted,gps,device,case_manager,cpims_ovc_id,household,0,variable,value
0,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282788.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,16142843-84ab-4cbe-9edd-de02ac81eef9,UnIque ID,2022-09-12-105151
1,9325,2022-09-12,105151,2022-09-27 12:49:56,3180741,Data Assistant,MAKE ME SMILE,ZILPA AKINYI,2022-09-27 13:49:49,2022-09-27 12:49:56,NaN,NaN,screen=1440 x 900;browser=Chrome;browserVersio...,NaN,4282807.0,ae1ff881-bf28-471a-9a03-3efceed4d96a,785a1f6b-6301-40dd-ac67-72b5f059d121,UnIque ID,2022-09-12-105151


In [12]:
cpara_unpivot.columns

Index(['record_id', 'assessment_date', 'cpims_id', 'entry_time', 'action_by',
       'designation', 'lip', 'entry_by_name', 'client_time', 'modified',
       'deleted', 'gps', 'device', 'case_manager', 'cpims_ovc_id', 'household',
       'variable', 'value'],
      dtype='object')

In [21]:
# filters bencmark questions and pivots them.
ovc_care_benchmark = cpara_unpivot[cpara_unpivot["variable"].str.contains('benchmark')].pivot(index=['record_id', 'cpims_id','assessment_date','entry_time', 'modified', 'cpims_ovc_id', 'household', 0], columns='variable', values=['value'])
# ovc_care_benchmark.to_excel("ovc_care_benchmark.xlsx")
ovc_care_benchmark.head(2)

value  \
variable                                                                                                                                                          benchmark1   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household                            0                                                 
240       2238085  2022-06-27      2022-09-09 15:35:53 2022-11-10 08:47:52 2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 383c621f-38f2-4db0-9409-5dcdcf452db9        Yes   
                                                                           3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 fcb63e98-0e12-4b59-b867-9c6e809b641f        Yes   

                                                                                                                                                                              \
variable                                                                                                                                                          benchmark2   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household                            0                                                 
240       2238085  2022-06-27      2022-09-09 15:35:53 2022-11-10 08:47:52 2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 383c621f-38f2-4db0-9409-5dcdcf452db9        Yes   
                                                                           3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 fcb63e98-0e12-4b59-b867-9c6e809b641f        Yes   

                                                                                                                                                                              \
variable                                                                                                                                                          benchmark3   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household                            0                                                 
240       2238085  2022-06-27      2022-09-09 15:35:53 2022-11-10 08:47:52 2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 383c621f-38f2-4db0-9409-5dcdcf452db9        Yes   
                                                                           3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 fcb63e98-0e12-4b59-b867-9c6e809b641f        Yes   

                                                                                                                                                                              \
variable                                                                                                                                                          benchmark4   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household                            0                                                 
240       2238085  2022-06-27      2022-09-09 15:35:53 2022-11-10 08:47:52 2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 383c621f-38f2-4db0-9409-5dcdcf452db9        Yes   
                                                                           3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1 fcb63e98-0e12-4b59-b867-9c6e809b641f        Yes   

                                                                                                                                                                              \
variable                                                                                                                                                          benchmark5   
record_id cpims_id assessment_date entry_time          modified            cpims_ovc_id household                            0                                                 
240       2238085  2022-06-27      2022-09-09 15:35:53 2022-11-10 08:47:52 2216746.0    f40fc0db-6b84-4c61-b998-a6e

In [28]:
# Filter cpara question and pivots them or not.  // send to excel
#ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')]
ovc_care_cpara = cpara_unpivot[cpara_unpivot["variable"].str.startswith('q')].pivot(index=['record_id', 'cpims_id','assessment_date','cpims_ovc_id', 'household'], columns='variable', values=['value'])
ovc_care_cpara.to_csv("ovc_care_cpara_pivot.csv")
ovc_care_cpara.head(2)

value  \
variable                                                                              q1_1   
record_id cpims_id assessment_date cpims_ovc_id household                                    
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1    na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1    na   

                                                                                           \
variable                                                                             q1_2   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q1_3   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q1_4   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1  Yes   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1  Yes   

                                                                                           \
variable                                                                             q2_1   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q2_2   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q2_3   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q2_4   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   
                                   3664650.0    f40fc0db-6b84-4c61-b998-a6ef9975fab1   na   

                                                                                           \
variable                                                                             q2_5   
record_id cpims_id assessment_date cpims_ovc_id household                                   
240       2238085  2022-06-27      2216746.0    f40fc0db-6b84-4c61-b998-a

In [10]:
cpara_individual.to_excel("cpara_ovc_questions.xlsx")
cpara_individual.head(2)

,record_id,cpara_record_id,cpara_date,question,response,cpims_id,caregiver_id,chv_id,cpims_ovc_id,household
0,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1687650.0,b540b485-7399-4407-bd14-066f5b79c133
1,5,8859,2022-08-02,q3.1,Yes,1705501,1676623,2042693,1705501.0,b540b485-7399-4407-bd14-066f5b79c133


In [11]:
cpims_hhs.columns

Index(['cpims_ovc_id', 'caregiver_id', 'household'], dtype='object')

In [108]:
cpara_data.to_excel("CPARA_MAPPING.xlsx")

In [109]:
cpara_data.head(2)

,UnIque ID,record_id,assessment_date,cpims_id,case_manager,child_headed,has_hei,has_pbf,has_svac,q1_1,...,cbo_id,cbo,county,subcounty,ward,chv_id,chv,cpims_ovc_id,caregiver_id,household
0,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282788.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
1,2022-09-12-105151,9325,2022-09-12,105151,NaN,No,No,No,No,Yes,...,2796.0,MAKE ME SMILE,Kisumu,Kisumu East,Manyatta 'B',3173240.0,GILLIANN GILLIAN ACHIENGGG,4282807.0,105151.0,ae1ff881-bf28-471a-9a03-3efceed4d96a
